In [2]:
pip install text-generation

  Obtaining dependency information for text-generation from https://files.pythonhosted.org/packages/5d/80/795be78bc14ab7ddc65620175508791747c13be11fa4848cef96c12012fa/text_generation-0.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic<2.0,>=1.10 from https://files.pythonhosted.org/packages/bc/e0/0371e9b6c910afe502e5fe18cc94562bfd9399617c7b4f5b6e13c29115b3/pydantic-1.10.12-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.9 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
def format_prompt_llama(instruction, preamble=None):
    template = """<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
<</SYS>>

{instruction} [/INST]"""
    if preamble is not None:
        return template.format(instruction=instruction, preamble=preamble)
    else:
        return template.format(instruction=instruction, preamble='')

    
def format_prompt_mpt(instruction):
    template = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n###Instruction\n{instruction}\n\n### Response\n"
    return template.format(instruction=instruction)

In [2]:
from text_generation import Client, AsyncClient

client = Client("http://127.0.0.1:8081")

client.generate('Hi!', max_new_tokens=512, temperature=0.7).generated_text

' I’m Sarah and I’m here to help you get off the computer and into the real world!\nGet ready to feel better, look better, and live better!\nJoin me in person or online for your free workshop to learn how to use essential oils, and I’ll teach you how to apply them to your daily life in a simple and effective way. I’ll also show you how to get started on your own, with a free starter kit!\nIf you’re interested in learning more about essential oils and how to use them, I’d love to help!\nThis program is completely free and open to anyone, and no experience with oils or aromatherapy is necessary.\nUncover the 5 benefits of essential oils, their history, and common uses.\nLearn the simple, 5-step process for applying essential oils to your daily life.\nSee how to get started with your own free starter kit!\nGet access to a private Facebook group where you can connect with other people who are using essential oils in their daily lives.\nPlus, get a free “Essential Oils for Beginners” guideb

In [6]:
import jsonlines
from tqdm import tqdm
import tqdm
from torch.utils.data import Dataset

with jsonlines.open('./prompts_augmented_v2_part2.jsonl') as reader:
    prompts = [row for row in reader]
    

    
continuations = []
# for row in tqdm(prompts):]

import asyncio
from typing import Coroutine, List, Sequence


def _limit_concurrency(
    coroutines: Sequence[Coroutine], concurrency: int
) -> List[Coroutine]:
    """Decorate coroutines to limit concurrency.
    Enforces a limit on the number of coroutines that can run concurrently in higher
    level asyncio-compatible concurrency managers like asyncio.gather(coroutines) and
    asyncio.as_completed(coroutines).
    """
    semaphore = asyncio.Semaphore(concurrency)

    async def with_concurrency_limit(coroutine: Coroutine) -> Coroutine:
        async with semaphore:
            return await coroutine

    return [with_concurrency_limit(coroutine) for coroutine in coroutines]


class ListDataset(Dataset):
    def __init__(self, original_list):
        self.original_list = original_list

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        return self.original_list[i]

    
from text_generation import Client

client = AsyncClient("http://127.0.0.1:8081", timeout=600)

ds = [format_prompt_mpt(row['prompt']) for row in prompts]


fs = [client.generate(prompt, max_new_tokens=512, temperature=0.7) for prompt in ds][:10]
fs = _limit_concurrency(fs,4)


sequences = await tqdm.asyncio.tqdm_asyncio.gather(*fs, total=len(fs))


for x, row in zip(sequences, prompts):
    continuation = x.generated_text
    continuations.append({**row, 'response': continuation})
    

    
with jsonlines.open('./output_augmented_v2_part2_mpt30instruct.jsonl','w') as writer:
    writer.write_all(continuations)

/var/tmp/ipykernel_19414/2464867425.py:53: RuntimeWarning: coroutine 'AsyncClient.generate' was never awaited
  fs = [client.generate(prompt, max_new_tokens=512, temperature=0.7) for prompt in ds][:10]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:46<00:00,  4.63s/it]


In [8]:
continuations

[{'dataset': 'amazon',
  'prompt': 'Given a item title and item specifications, this program will generate exciting product descriptions.\n\nMelrose International Frosted Winterberry Wreath, 26-Inch\n\nItem specifications:\nCedar, with laurel leaves and clumps of white berries with just a light dusting of snow\nAll stems are wired for easy shaping to full size\nSuitable for use inside or outside in a protected area\nThis wreath adds a touch of warmth to any room or door\n\nRespond using complex language, long words and technical terms, as if you are an expert.\nProduct description:',
  'sample_ix': 50,
  'original_prompt': 'Given a item title and item specifications, this program will generate exciting product descriptions.\n\nMelrose International Frosted Winterberry Wreath, 26-Inch\n\nItem specifications:\nCedar, with laurel leaves and clumps of white berries with just a light dusting of snow\nAll stems are wired for easy shaping to full size\nSuitable for use inside or outside in a 

Task exception was never retrieved
future: <Task finished name='Task-340' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at /opt/conda/envs/py310/lib/python3.10/site-packages/tqdm/asyncio.py:75> exception=ClientOSError(104, 'Connection reset by peer')>
Traceback (most recent call last):
  File "/opt/conda/envs/py310/lib/python3.10/site-packages/tqdm/asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "/var/tmp/ipykernel_19414/3316032549.py", line 30, in with_concurrency_limit
    return await coroutine
  File "/opt/conda/envs/py310/lib/python3.10/site-packages/text_generation/client.py", line 385, in generate
    async with session.post(self.base_url, json=request.dict()) as resp:
  File "/opt/conda/envs/py310/lib/python3.10/site-packages/aiohttp/client.py", line 1141, in __aenter__
    self._resp = await self._coro
  File "/opt/conda/envs/py310/lib/python3.10/site-packages/aiohttp/client.py", line 560, in _request
    await resp.start(conn)
  File "